In [18]:
import sys
import bqplot
import numpy as np
import pandas as pd
import ipywidgets as widgets
from restart import RestartModel
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from bqplot import pyplot as plt


chart_colors = ["#77AADD", "#99DDFF", "#44BB99", "#BBCC33", "#AAAA00", "#EEDD88",
                "#EE8866", "#FFAABB", "#DDDDDD", "#000000"]

def generate_bar(df, title="", scientific_notation=False, small_xlabel=False):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    bar_chart = plt.bar(x=x_titles,
                        y=df,
                        colors=chart_colors[:df.index.values.size])
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_group_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="grouped",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_scatter(df, title="", scientific_notation=False, small_xlabel=True):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    scatter = plt.scatter(x=x_titles, y=df)
    
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig
    
def generate_stacked_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="stacked",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_separate_bar_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    bar_list = []
    for col in df.columns: # .values.tolist()
        bar_list.append(generate_bar(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return bar_list

def generate_separate_scatter_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    scatter_list = []
    for col in df.columns: # .values.tolist()
        scatter_list.append(generate_scatter(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return scatter_list

def generate_html_legend(df, colors=chart_colors, table=True, font_size=13):
    name = df.index.name.rsplit(" ", 1)[0]
    html_string = f"<div style='font-size:{font_size}px; font-family:helvetica'><b style='font-weight:bold'>{name}</b>"
    indices = df.index.values
    if table:
        html_string += "<table><tr>"
        for i in range(0, indices.size):
            if i % 2 == 0:
                index_num = int(i / 2)
            else:
                index_num = int(i / 2 + indices.size / 2)
            html_string += f"<td style='padding:0 5px'><span style='color:{colors[index_num]}'>█</span> {indices[index_num]}</td>"
            if i % 2 != 0:
                html_string += "</tr><tr>"
        html_string += "</tr></table>"
    else:
        for_count = 0
        for string in indices:
            if for_count == 0:
                html_string += "<br>"
            else:
                html_string += "&emsp;"
            html_string += f"<span style='color:{colors[for_count]}'>█</span> {string}"
            for_count += 1
    html_string += "</div>"
    return widgets.HTML(html_string)

def generate_group_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_group_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

def generate_stacked_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_stacked_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

In [40]:
restart = RestartModel(
    population='oes',
    state='Washington',
    county='King',
)

model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))

burn_chart = generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)

out = widgets.interactive_output(dashboard, {'backstop': slider})

In [54]:
restart_health = RestartModel(
    population='oes',
    state='Washington',
    county='King',
    subpop='healthcare'
)

model_health = restart_health.model
model_health.inventory.set_average_orders_per_period(model_health.demand.demand_by_popsum1_total_rp1n_tc)

slider_health = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard_h(backstop):
    set_h_stock(backstop)
    
def display_h_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_h_stock(backstop):
    backstop = [backstop]
    model_health.inventory.order(model_health.inventory.inv_by_popsum1_total_rp1n_tc)
    model_health.inventory.set_min_in_periods(backstop)
    display_h_stock(model_health.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model_health.demand.demand_per_unit_map_dn_um.df))
pop_health = format_population(to_sheet(model_health.population.population_pP_tr.df))

health_out = widgets.interactive_output(dashboard_h, {'backstop': slider_health})

In [58]:
config_2 = set_config('config/default')
restart_2 = RestartModel(population='oes', state='Washington', subpop='wa_tier2_opt2')
model_2 = restart_2.model
model_2.inventory.set_average_orders_per_period(model_2.demand.demand_by_popsum1_total_rp1n_tc)

slider_2 = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard_2(backstop):
    set_stock_2(backstop)
    
def display_stock_2(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock_2(backstop):
    backstop = [backstop]
    model_2.inventory.order(model_2.inventory.inv_by_popsum1_total_rp1n_tc)
    model_2.inventory.set_min_in_periods(backstop)
    display_stock_2(model_2.inventory.inv_by_popsum1_total_rp1n_tc.df)

pop_2 = format_population(to_sheet(model_2.population.population_pP_tr.df))
    
out_2 = widgets.interactive_output(dashboard_2, {'backstop': slider_2})

In [59]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['PPE Analysis']),
    v.Tab(children=['Burn Rates']),
    v.Tab(children=['General Population']),
    v.Tab(children=['Healthcare Workers']),
    v.Tab(children=['Outpatient/First Responders']),
                
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["PPE Stockpile"]),
                v.CardText(children=[
                    "General population:"
                ]),
                slider, out,
                v.CardText(children=[
                    "Healthcare workers:"
                ]),
                slider_health, health_out,
                v.CardText(children=[
                    "Outpatient healthcare workers and first responders"
                ]),
                slider_2, out_2,
            ])
                
        ])
    ]),
                
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand"]),
                wa_burn_sheet, burn_chart
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["General Population by SOC Code"]),
                v.CardSubtitle(children=["Population Breakdown"]),
                pop,
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Healthcare Workers by SOC Code"]),
                v.CardSubtitle(children=["Population Breakdown"]),
                pop_health,
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Outpatient Healthcare and First Responders by SOC Code"]),
                v.CardSubtitle(children=["Population Breakdown"]),
                pop_2,
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['PPE Analysis']), Tab(children=['Burn Rates']), Tab(children=['General Population…